<h2>Import Libraries

In [18]:
from splinter import Browser
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unicodedata import normalize
from bs4 import BeautifulSoup
import tweepy

<h2>Establish the urls that will be used to gather NFT data including lists and pricing data

In [2]:
url = 'https://rarity.tools/'
urls = ['https://www.coingecko.com/en/nft?page=2','https://www.coingecko.com/en/nft?page=3','https://www.coingecko.com/en/nft?page=4']

<h2>Set up the Splinter Instance Using Chrome
<h3>Splinter is a highly abstracted version of Selenium, which is an HTML navigating tool for Python. The beauty of splinter is you can extract any tag, css, iframe, or other otherwise difficult to reach part of a website and it works with pandas

In [3]:
browser = Browser('chrome')

In [5]:
browser.visit(url)

<h2> Collect the HTML of the website to parse with beautiful soup and/or with PANDAS.read_html

In [6]:
html = browser.html

In [7]:
soup = BeautifulSoup(html, 'html.parser')

<h2>In this case, we want to download the entire first Rarity.tools table (top NFT collections), which PANDAS does with read_html()

In [8]:
table_list = pd.read_html(str(soup.find_all('table')[0]))

<h2>Because of the way we've used Beautiful Soup to find the correct table, Read_html actually creates a list of dataframes, one for each table on that page of the site... We want the first one in this case

In [44]:
df = table_list[0]

<h2>Set index to the order number provided by Rarity tools

In [45]:
df = df.set_index('#')

In [46]:
df.head(10)

,Collection,Volume (7d),Sales (7d),⏷ Volume (All Time),Sales (All Time),Avg Price (7d),Total Supply,Owners,Owners%,Estimated Market Cap,Added
#,,,,,,,,,,,
1,CryptoPunks,"6,710.94 ETH",83,"733,379.72 ETH",17941,80.85 ETH,9999,3250,32.50%,"808,466.13 ETH",2021/04/30
2,Bored Ape YC,"7,864.43 ETH",145,"248,835.82 ETH",22967,54.24 ETH,10000,5908,59.08%,"542,374.31 ETH",2021/05/02
3,Mutant Ape Yacht Club,"3,444.18 ETH",461,"123,312.66 ETH",18216,7.47 ETH,16899,10448,61.83%,"126,254.07 ETH",2021/08/29
4,Meebits,882.23 ETH,178,"76,481.09 ETH",13999,4.96 ETH,19999,5332,26.66%,"99,122.14 ETH",2021/05/04
5,Loot (for Adventurers),274.20 ETH,70,"71,396.42 ETH",10455,3.92 ETH,7779,2539,32.64%,"30,471.93 ETH",2021/09/04
6,Cool Cats NFT,845.39 ETH,105,"58,863.91 ETH",23375,8.05 ETH,9933,5013,50.47%,"79,973.46 ETH",2021/07/02
7,CrypToadz by GREMPLIN,583.07 ETH,154,"49,723.57 ETH",15390,3.79 ETH,7025,3539,50.38%,"26,597.74 ETH",2021/09/11
8,0N1 Force,77.47 ETH,82,"44,618.13 ETH",16357,0.94 ETH,7775,4208,54.12%,"7,345.39 ETH",2021/08/22
9,MekaVerse,239.30 ETH,193,"43,438.17 ETH",9706,1.23 ETH,8887,5091,57.29%,"10,940.54 ETH",2021/10/16


<h2>Convert the <em>Collection</em> column to a list so we can cycle through the first 100 projects for  Twitter sentiment data

In [47]:
collection_list = df['Collection'].to_list()

In [48]:
len(collection_list)

598

<h2>Create a new list of the top 100 NFT collections

In [15]:
top_NFT_collection_list = collection_list[:100]

In [16]:
top_NFT_collection_list

['CryptoPunks',
 'Bored Ape YC',
 'Mutant Ape Yacht Club',
 'Meebits',
 'Loot (for Adventurers)',
 'Cool Cats NFT',
 'CrypToadz by GREMPLIN',
 '0N1 Force',
 'MekaVerse',
 'Pudgy Penguins',
 'Hashmasks',
 'Bored Ape Kennel Club',
 'CyberKongz',
 'VeeFriends',
 'World of Women',
 'The Doge Pound',
 'Sneaky Vampire Syndicate',
 'Lazy Lions',
 'Axie Origins',
 'CyberKongz VX',
 'SupDucks',
 'FLUF World',
 'JUNGLE FREAKS BY TROSLEY',
 'GalacticApes',
 'MutantCats',
 'Gutter Cat Gang',
 'Vox Collectibles',
 'MoonCats',
 'Bears Deluxe',
 'Koala Intelligence Agency',
 'The Sevens (Official)',
 'Adam Bomb Squad',
 'The Humanoids',
 'PEACEFUL GROUPIES',
 'The CryptoDads',
 'Rumble Kong League',
 'Robotos Official',
 'THE SHIBOSHIS',
 'Animetas',
 'DeadFellaz',
 'The Wicked Craniums',
 'GEVOLs.',
 'Winter Bears',
 'The Vogu Collective',
 'FVCK_CRYSTAL//',
 'Metasaurs by Dr. DMT',
 'Wicked Ape Bone Club',
 'BASTARD GAN PUNKS V2',
 'Stoner Cats',
 'Cupcats Official',
 'Gambling Apes',
 'Ape Gang',


<h2>Using the Rarity.Tools list of top 100 NFT projects, we connect the Twitter API to collect 100 tweets for each name in the NFT collection to analyze for sentiment

In [19]:
# Twitter API Dependencies
import tweepy

# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Twitter API Keys
from config import (consumer_key,
                    consumer_secret,
                    access_token,
                    access_token_secret)

# Setup Tweepy API Authentication
# api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())
api = tweepy.API(auth, wait_on_rate_limit=True)
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [28]:
# List for dictionaries of results
public_tweets = []
results_list = []

# Loop through each user
for nft in top_NFT_collection_list:

    # Get all tweets from home feed
    for tweet in tweepy.Cursor(api.search_tweets, nft).items(100):
        public_tweets.append(tweet.text)
        
        # Sentiment lists for holding sentiment scores for each tweet 
        compound_list = []
        positive_list = []
        negative_list = []
        neutral_list = []

        # Loop through all tweets for each NFT
        for tweet in public_tweets:

            # Run Vader Analysis on each tweet
            results = analyzer.polarity_scores(tweet)
            compound = results["compound"]
            pos = results["pos"]
            neu = results["neu"]
            neg = results["neg"]

            # Add each value to the appropriate list
            compound_list.append(compound)
            positive_list.append(pos)
            negative_list.append(neg)
            neutral_list.append(neu)

    # Create a dictionaty of results - One for each NFT Project
    nft_results = {
        "NFT_Collection": nft,
        "Compound Score": np.mean(compound_list),
        "Postive Score": np.mean(positive_list),
        "Neutral Score": np.mean(neutral_list),
        "Negative Score": np.mean(negative_list)
        }
    #Create a master list of the 100 Projects with a dict holding the mean Vadar scores for each project's Pos Neg Cmp Neu
    results_list.append(nft_results)    


Rate limit reached. Sleeping for: 78


<h2>Create a DataFrame out of the resulting list of dicts

In [50]:
vadar_df = pd.DataFrame(results_list)

In [52]:
vadar_df.head()

,NFT_Collection,Compound Score,Postive Score,Neutral Score,Negative Score
0,CryptoPunks,0.272199,0.139650,0.830140,0.030240
1,Bored Ape YC,0.070356,0.089370,0.851575,0.059070
2,Mutant Ape Yacht Club,0.116763,0.085500,0.873793,0.040720
3,Meebits,0.104867,0.076433,0.884758,0.038815
4,Loot (for Adventurers),0.144161,0.088358,0.875068,0.036578


<h2>Sort by Compound Score to find the overall highest sentiment per project

In [55]:
vadar_df = vadar_df.sort_values(by=['Compound Score'], ascending=False)
vadar_df.head(10)

,NFT_Collection,Compound Score,Postive Score,Neutral Score,Negative Score
0,CryptoPunks,0.272199,0.139650,0.830140,0.030240
5,Cool Cats NFT,0.204639,0.106455,0.859017,0.034530
16,Sneaky Vampire Syndicate,0.180117,0.107525,0.850559,0.041941
24,MutantCats,0.179882,0.107013,0.850773,0.042223
25,Gutter Cat Gang,0.174589,0.104116,0.854268,0.041625
23,GalacticApes,0.169031,0.103855,0.852642,0.043513
15,The Doge Pound,0.166407,0.103721,0.854613,0.041694
18,Axie Origins,0.165816,0.104499,0.849041,0.046481
21,FLUF World,0.165122,0.103920,0.852673,0.043424
19,CyberKongz VX,0.164002,0.102567,0.852727,0.044726


<h2>Because of the rate limitations with Twitter's API, we send the DF to a CSV since it takes several hours to collect the data

In [41]:
vadar_df.to_csv('NFTCollection.csv')